In [1]:
import MagmaPandas.geochemistry.volatile_solubility.Allison2022 as a
import MagmaPandas.geochemistry.volatile_solubility.IaconoMarziano as im
from MagmaPandas.geochemistry.eos_volatiles import hollowayBlank as h
a_config = a.Allison_configuration()
a_config.model = "mixed"
a_config.print()


## Allison (2022) volatile solubility ##
Settings________________________________
Fugacity model.............hollowayBlank
Species model......................mixed

Calibration range______________________
Temperature..................1273-1673°K
Pressure........................< 7 kbar


In [2]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.parse.readers as r
import pandas as pd
import elements as e
import numpy as np
import matplotlib.pyplot as plt
import VESIcal as v
import meltInc.plotting as p
config = mp.configuration()

p.layout()

/Users/thomas/miniforge3/lib/python3.9/site-packages/VESIcal/calculate_classes.py:7: UserWarning: 

  from VESIcal.models import magmasat


In [3]:
config.print()

Kd Fe-Mg ol-melt.......................toplis
Melt Fe3+/Fe2+........................borisov
Melt thermometer...............putirka2008_14
Volatile solubility model......IaconoMarziano


In [4]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
inclusion = melt.iloc[10, :].copy()
inclusion

SiO2     46.889000
Al2O3    15.078000
MgO       6.831000
CaO      12.663000
FeO       7.380000
Na2O      3.146000
K2O       1.766000
MnO       0.135000
TiO2      2.642000
P2O5      0.972000
Cr2O3     0.055000
CO2       0.280800
H2O       1.070768
F         0.072400
S         0.134300
Cl        0.058700
total    99.173965
dtype: float32

In [5]:
P_bar = 3e3
T_K = 1200+273.15

In [7]:
im.calculate_saturation(inclusion, T_K), a.calculate_saturation(inclusion, T_K)

(2773.2839565437334, 2584.299581634737)

In [20]:
a.calculate_solubility(inclusion, P_bar=2584.29958163, T_K=T_K, x_fluid=0.00)

(0, 0.2808000246941415)

In [ ]:
Xfl = np.arange(0,1,0.01)
Ca = []
Cim = []

for x in Xfl:
    C = a.calculate_solubility(inclusion, P_bar, T_K, x_fluid=x, output="CO2")
    Ca.append(C)


plt.plot(Xfl, Ca)



In [ ]:
i

In [ ]:
im.calculate_solubility(inclusion, P_bar, T_K, x_fluid=0.6, output="CO2")

In [ ]:
a.calculate_solubility(inclusion, P_bar, T_K, x_fluid=0.05, output="CO2")

In [ ]:
inclusion.index.difference(inclusion._weights.index)

In [ ]:
CO2 = inclusion["CO2"]
cations = a.co2._cation_fractions_Allison(inclusion)
deltaV = a.co2._deltaV(cations)
lnK0 = a.co2._lnK0(cations)
FW = 36.594  # alkali basalt formula weight per 1 oxygen

XCO3 = CO2 * (1 / 44.01) / ((100 / FW) - (CO2 / FW))
Kf = XCO3 / (1 + XCO3)
deltaV, lnK0, XCO3

In [ ]:
a.co2._root_partial_pressure(2e3, T_K, deltaV, lnK0, Kf)

In [ ]:
pressures = np.arange(0, 7.250e3, 250)

H2O = [a.h2o.calculate_solubility(p, T_K) for p in pressures]
CO2 = [a.co2.calculate_solubility(inclusion, p, T_K) for p in pressures]
CO2im = [im.co2.calculate_solubility(inclusion, p, T_K) for p in pressures]
H2Oim = [im.h2o.calculate_solubility(inclusion, p, T_K) for p in pressures]

fig, ax = plt.subplots(figsize=(10,10))

ax.plot(pressures, CO2)
ax.plot(pressures, CO2im, label="i-m")
plt.legend()
plt.show()

In [ ]:
pd.DataFrame({"P":pressures, "CO2":CO2, "CO2im": CO2im})

In [ ]:
pressures